<a href="https://colab.research.google.com/github/CaioPassos3/AprendizagemDeMaquina/blob/main/Lista3_AprendizagemDeMaquina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from numpy.linalg import inv
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict

Questão 1.

Considere o conjunto de dados disponível em kc2.csv, organizado em 22 colunas,
sendo as 21 primeiras colunas os atributos e a última coluna a saída. Os 21
atributos são referentes à caracterização de códigos-fontes para processamento
de dados na NASA. A saída é a indicação de ausência (0) ou existência (1) de
defeitos (os dados foram balanceados via subamostragem). Maiores detalhes
sobre os dados podem ser conferidos em https://www.openml.org/search?
type=data&sort=runs&id=1063&status=active.

In [17]:
file_path = "kc2.csv"
df = pd.read_csv(file_path)

df.head()

,4.000000000000000000e+00,1.000000000000000000e+00,1.000000000000000000e+00.1,1.000000000000000000e+00.2,4.000000000000000000e+00.1,8.000000000000000000e+00,6.700000000000000400e-01,1.500000000000000000e+00,5.330000000000000071e+00,1.200000000000000000e+01,...,2.000000000000000000e+00,0.000000000000000000e+00.1,0.000000000000000000e+00.2,0.000000000000000000e+00.3,3.000000000000000000e+00,1.000000000000000000e+00.3,3.000000000000000000e+00.1,1.000000000000000000e+00.4,1.000000000000000000e+00.5,0.000000000000000000e+00.4
0,39.0,4.0,1.0,2.0,105.0,520.19,0.07,13.89,37.44,7227.91,...,29.0,1.0,4.0,2.0,12.0,19.0,61.0,44.0,7.0,0.0
1,1.0,1.0,1.0,1.0,6.0,15.51,0.40,2.50,6.20,38.77,...,0.0,0.0,0.0,0.0,5.0,1.0,5.0,1.0,1.0,0.0
2,15.0,1.0,1.0,1.0,55.0,224.81,0.17,5.73,39.25,1287.55,...,12.0,0.0,1.0,0.0,6.0,11.0,34.0,21.0,1.0,0.0
3,12.0,2.0,1.0,2.0,15.0,45.00,0.17,6.00,7.50,270.00,...,8.0,1.0,0.0,0.0,6.0,2.0,11.0,4.0,3.0,0.0
4,8.0,1.0,1.0,1.0,13.0,43.19,0.27,3.75,11.52,161.94,...,6.0,0.0,0.0,0.0,6.0,4.0,8.0,5.0,1.0,0.0


a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação
binária nos dados em questão. Para tanto, use as abordagens
abaixo:

– KNN (escolha k = 1 e k = 5, distância Euclidiana e Mahalonobis,
totalizando 4 combinações);

– Árvore de decisão (você pode usar uma implementação já existente,
como a do scikit-learn, com índices de impureza de gini e entropia).

In [18]:
# Renomeando colunas: attr1, attr2, ..., attr21, target
df.columns = [f'attr{i+1}' for i in range(21)] + ['target']

# Separando atributos e rótulo
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Normalizando os dados para evitar viés em distâncias
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Preparando a divisão para validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

X_scaled[:5], y[:5]  # Exibindo amostra dos dados processados

(array([[-0.18177751, -0.23335236, -0.28355717, -0.30306409, -0.15475783,
         -0.16971078, -0.5897163 ,  0.02376097, -0.03610969, -0.1777248 ,
         -0.17807838, -0.16100217, -0.17518265, -0.32803458, -0.23067817,
          1.0036755 ,  0.08973792, -0.09251015, -0.17006832, -0.14032048,
         -0.22975364],
        [-0.51861523, -0.41835504, -0.28355717, -0.38635783, -0.44442395,
         -0.3549343 ,  0.89919026, -0.79240457, -0.79441995, -0.2190013 ,
         -0.35409088, -0.19524102, -0.48364994, -0.45887984, -0.54564258,
         -0.32416849, -0.89408   , -0.68962115, -0.43698607, -0.46560403,
         -0.41477264],
        [-0.39451712, -0.41835504, -0.28355717, -0.38635783, -0.30105385,
         -0.27811873, -0.13853249, -0.56095464,  0.0078257 , -0.21183142,
         -0.28808619, -0.18929337, -0.3560083 , -0.45887984, -0.46690148,
         -0.32416849, -0.75353459, -0.35789282, -0.2987608 , -0.31430936,
         -0.41477264],
        [-0.42110957, -0.35668748, -0.28355

In [23]:
# Carregando o dataset
df = pd.read_csv('kc2.csv', header=None)
df.columns = [f'attr{i+1}' for i in range(21)] + ['target']

# Separando atributos e rótulo
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definindo o validador 10-fold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Avaliando os modelos
results = {}

for k in [1, 5]:
    # Euclidiana
    accs = []
    for train_idx, test_idx in cv.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accs.append(accuracy_score(y_test, y_pred))
    results[f'KNN (k={k}, Euclidiana)'] = accs

    # Mahalanobis
    accs = []
    for train_idx, test_idx in cv.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        VI = inv(np.cov(X_train.T))
        knn = KNeighborsClassifier(n_neighbors=k, metric='mahalanobis', metric_params={'VI': VI})
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accs.append(accuracy_score(y_test, y_pred))
    results[f'KNN (k={k}, Mahalanobis)'] = accs

# Exibindo resultados
print("=== Resultados KNN (Acurácia Média ± Desvio Padrão) ===")
for name, scores in results.items():
    print(f"{name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

=== Resultados KNN (Acurácia Média ± Desvio Padrão) ===
KNN (k=1, Euclidiana): 0.7677 ± 0.1104
KNN (k=1, Mahalanobis): 0.6931 ± 0.1048
KNN (k=5, Euclidiana): 0.7998 ± 0.0765
KNN (k=5, Mahalanobis): 0.7299 ± 0.0647


In [22]:
# Avaliando as árvores de decisão
tree_results = {}

for criterion in ['gini', 'entropy']:
    accs = []
    for train_idx, test_idx in cv.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        tree = DecisionTreeClassifier(criterion=criterion, random_state=42)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        accs.append(accuracy_score(y_test, y_pred))
    tree_results[f'Tree ({criterion})'] = accs

# Exibindo resultados
print("=== Resultados Árvore de Decisão (Acurácia Média ± Desvio Padrão) ===")
for name, scores in tree_results.items():
    print(f"{name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

=== Resultados Árvore de Decisão (Acurácia Média ± Desvio Padrão) ===
Tree (gini): 0.7024 ± 0.1212
Tree (entropy): 0.6937 ± 0.1472


b) Para cada modelo criado, reporte valor médio e desvio padrão das métricas
de acurácia, revocação, precisão e F1-score.

In [25]:
# Carregando os dados
df = pd.read_csv('kc2.csv', header=None)
df.columns = [f'attr{i+1}' for i in range(21)] + ['target']
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Avaliando os modelos
from collections import defaultdict
metrics = defaultdict(list)

for k in [1, 5]:
    for dist in ['euclidean', 'mahalanobis']:
        name = f"KNN (k={k}, {dist})"

        for train_idx, test_idx in cv.split(X_scaled, y):
            X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            if dist == 'mahalanobis':
                VI = inv(np.cov(X_train.T))
                model = KNeighborsClassifier(n_neighbors=k, metric='mahalanobis', metric_params={'VI': VI})
            else:
                model = KNeighborsClassifier(n_neighbors=k, metric='euclidean')

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            metrics[f'{name} - Accuracy'].append(accuracy_score(y_test, y_pred))
            metrics[f'{name} - Precision'].append(precision_score(y_test, y_pred))
            metrics[f'{name} - Recall'].append(recall_score(y_test, y_pred))
            metrics[f'{name} - F1'].append(f1_score(y_test, y_pred))

# Mostrando resultados
print("=== Resultados KNN ===")
for key in metrics:
    print(f"{key}: {np.mean(metrics[key]):.4f} ± {np.std(metrics[key]):.4f}")

=== Resultados KNN ===
KNN (k=1, euclidean) - Accuracy: 0.7677 ± 0.1104
KNN (k=1, euclidean) - Precision: 0.7979 ± 0.1352
KNN (k=1, euclidean) - Recall: 0.7418 ± 0.1506
KNN (k=1, euclidean) - F1: 0.7597 ± 0.1127
KNN (k=1, mahalanobis) - Accuracy: 0.6931 ± 0.1048
KNN (k=1, mahalanobis) - Precision: 0.7073 ± 0.1015
KNN (k=1, mahalanobis) - Recall: 0.6682 ± 0.1550
KNN (k=1, mahalanobis) - F1: 0.6810 ± 0.1143
KNN (k=5, euclidean) - Accuracy: 0.7998 ± 0.0765
KNN (k=5, euclidean) - Precision: 0.8141 ± 0.0968
KNN (k=5, euclidean) - Recall: 0.7955 ± 0.1154
KNN (k=5, euclidean) - F1: 0.7980 ± 0.0729
KNN (k=5, mahalanobis) - Accuracy: 0.7299 ± 0.0647
KNN (k=5, mahalanobis) - Precision: 0.7967 ± 0.0947
KNN (k=5, mahalanobis) - Recall: 0.6373 ± 0.1345
KNN (k=5, mahalanobis) - F1: 0.6971 ± 0.0850


In [27]:
# Avaliando as árvores
tree_metrics = defaultdict(list)

for criterion in ['gini', 'entropy']:
    name = f"Tree ({criterion})"

    for train_idx, test_idx in cv.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = DecisionTreeClassifier(criterion=criterion, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        tree_metrics[f'{name} - Accuracy'].append(accuracy_score(y_test, y_pred))
        tree_metrics[f'{name} - Precision'].append(precision_score(y_test, y_pred))
        tree_metrics[f'{name} - Recall'].append(recall_score(y_test, y_pred))
        tree_metrics[f'{name} - F1'].append(f1_score(y_test, y_pred))

# Mostrando resultados
print("=== Resultados Árvore de Decisão ===")
for key in tree_metrics:
    print(f"{key}: {np.mean(tree_metrics[key]):.4f} ± {np.std(tree_metrics[key]):.4f}")

=== Resultados Árvore de Decisão ===
Tree (gini) - Accuracy: 0.7024 ± 0.1212
Tree (gini) - Precision: 0.7307 ± 0.1363
Tree (gini) - Recall: 0.6482 ± 0.1516
Tree (gini) - F1: 0.6817 ± 0.1347
Tree (entropy) - Accuracy: 0.6937 ± 0.1472
Tree (entropy) - Precision: 0.7119 ± 0.1587
Tree (entropy) - Recall: 0.6591 ± 0.1741
Tree (entropy) - F1: 0.6803 ± 0.1557
